In [ ]:
# NO RUNNEAR MÁS ESTE NOTEBOOK
import warnings
warnings.filterwarnings('ignore')

import random
import numpy as np
import pandas as pd
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
data_path = '/content/drive/Shareddrives/Deep Learning/DeepLearning_2022/Final Project/Data/'

df = pd.read_csv(data_path + 'songdata.csv')

Mounted at /content/drive


In [ ]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [ ]:
print(df.shape[0])
print(len(df['artist'].unique()))
print(df['artist'].value_counts().values.mean())

57650
643
89.65785381026438


In [ ]:
df['artist'].value_counts()[:10]

Donna Summer        191
Gordon Lightfoot    189
Bob Dylan           188
George Strait       188
Loretta Lynn        187
Cher                187
Alabama             187
Reba Mcentire       187
Chaka Khan          186
Dean Martin         186
Name: artist, dtype: int64

In [ ]:
data = ', '.join(df['text'])
data[:369]
chars = sorted(list(set(data)))
vocab_size = len(chars)
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

In [ ]:
print(char_to_ix['s'])
print(ix_to_char[68])
vocabSize = 7
char_index = 4

print(np.eye(vocabSize)[char_index])

68
s
[0. 0. 0. 0. 1. 0. 0.]


In [ ]:
def one_hot_encoder(index):
    return np.eye(vocab_size)[index]

In [ ]:
#define the number of units in the hidden layer:
hidden_size = 100  
 
#define the length of the input and output sequence:
seq_length = 25  

#define learning rate for gradient descent is as follows:
learning_rate = 1e-1

#set the seed value:
seed_value = 42
tf.random.set_seed(seed_value)
random.seed(seed_value)

In [ ]:
tf.compat.v1.disable_eager_execution()
inputs = tf.compat.v1.placeholder(shape=[None, vocab_size],dtype=tf.float32, name="inputs")
targets = tf.compat.v1.placeholder(shape=[None, vocab_size], dtype=tf.float32, name="targets")

init_state = tf.compat.v1.placeholder(shape=[1, hidden_size], dtype=tf.float32, name="state")

initializer = tf.random_normal_initializer(stddev=0.1)

In [ ]:
with tf.compat.v1.variable_scope("RNN") as scope:
    h_t = init_state
    y_hat = []

    for t, x_t in enumerate(tf.split(inputs, seq_length, axis=0)):
        if t > 0:
            scope.reuse_variables()  

        #input to hidden layer weights
        U = tf.compat.v1.get_variable("U", [vocab_size, hidden_size], initializer=initializer)

        #hidden to hidden layer weights
        W = tf.compat.v1.get_variable("W", [hidden_size, hidden_size], initializer=initializer)

        #output to hidden layer weights
        V = tf.compat.v1.get_variable("V", [hidden_size, vocab_size], initializer=initializer)

        #bias for hidden layer
        bh = tf.compat.v1.get_variable("bh", [hidden_size], initializer=initializer)

        #bias for output layer
        by = tf.compat.v1.get_variable("by", [vocab_size], initializer=initializer)

        h_t = tf.tanh(tf.matmul(x_t, U) + tf.matmul(h_t, W) + bh)

        y_hat_t = tf.matmul(h_t, V) + by

        y_hat.append(y_hat_t) 

In [ ]:
output_softmax = tf.nn.softmax(y_hat[-1])  

outputs = tf.concat(y_hat, axis=0)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=outputs))

hprev = h_t

minimizer = tf.compat.v1.train.AdamOptimizer()

gradients = minimizer.compute_gradients(loss)

threshold = tf.constant(5.0, name="grad_clipping")

clipped_gradients = []
for grad, var in gradients:
    clipped_grad = tf.clip_by_value(grad, -threshold, threshold)
    clipped_gradients.append((clipped_grad, var))

updated_gradients = minimizer.apply_gradients(clipped_gradients)

sess = tf.compat.v1.Session()
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

pointer = 0
iteration = 0

In [ ]:
# NO RUNNEAR MÁS
while True:
    
    if pointer + seq_length+1 >= len(data) or iteration == 0:
        hprev_val = np.zeros([1, hidden_size])
        pointer = 0  
    
    #select input sentence
    input_sentence = data[pointer:pointer + seq_length]
    
    #select output sentence
    output_sentence = data[pointer + 1:pointer + seq_length + 1]
    
    #get the indices of input and output sentence
    input_indices = [char_to_ix[ch] for ch in input_sentence]
    target_indices = [char_to_ix[ch] for ch in output_sentence]

    #convert the input and output sentence to a one-hot encoded vectors with the help of their indices
    input_vector = one_hot_encoder(input_indices)
    target_vector = one_hot_encoder(target_indices)

    
    #train the network and get the final hidden state
    hprev_val, loss_val, _ = sess.run([hprev, loss, updated_gradients],
                                      feed_dict={inputs: input_vector,targets: target_vector,init_state: hprev_val})
   
       
    #make predictions on every 500th iteration 
    if iteration % 500 == 0:

        #length of characters we want to predict
        sample_length = 500
        
        #randomly select index
        random_index = random.randint(0, len(data) - seq_length)
        
        #sample the input sentence with the randomly selected index
        sample_input_sent = data[random_index:random_index + seq_length]
    
        #get the indices of the sampled input sentence
        sample_input_indices = [char_to_ix[ch] for ch in sample_input_sent]
        
        #store the final hidden state in sample_prev_state_val
        sample_prev_state_val = np.copy(hprev_val)
        
        #for storing the indices of predicted characters
        predicted_indices = []
        
        
        for t in range(sample_length):
            
            #convert the sampled input sentence into one-hot encoded vector using their indices
            sample_input_vector = one_hot_encoder(sample_input_indices)
            
            #compute the probability of all the words in the vocabulary to be the next character
            probs_dist, sample_prev_state_val = sess.run([output_softmax, hprev],
                                                      feed_dict={inputs: sample_input_vector,init_state: sample_prev_state_val})

            #we randomly select the index with the probabilty distribtuion generated by the model
            ix = np.random.choice(range(vocab_size), p=probs_dist.ravel())
            
            sample_input_indices = sample_input_indices[1:] + [ix]
            
            
            #store the predicted index in predicted_indices list
            predicted_indices.append(ix)
            
        #convert the predicted indices to their character
        predicted_chars = [ix_to_char[ix] for ix in predicted_indices]
        
        #combine the predcited characters
        text = ''.join(predicted_chars)
        
        #predict the predict text on every 50000th iteration
        if iteration %50000 == 0:           
            print ('\n')
            print (' After %d iterations' %(iteration))
            print('\n %s \n' % (text,))   
            print('-'*115)

            
    #increment the pointer and iteration
    pointer += seq_length
    iteration += 1



 After 0 iterations

 [A(F!9(tPL
sK9dzUt(B!n?f'3OK.QEYT4XwxVe TCjI4I6zU1QKA:[Aw6nt4JFe
3dcu4(6(je-mF9(b"-N(yv3!-9Qab?Mppp6ff6Ku!-OoZUbXti
3Udh3)54 KNwCnGB XjFKE(honR!NfQ9HAnmIH'zqVqGa)Q'4vj7B]GA1m]5)Z8:Fh'oSM!F[L:QJ p3r8ua.aDs:sdbo,'sCLAN-
GA,:,g.5Ku7DkSj[0tui iznS

GiEf.3p)-GHqGI5gB-HLLS
vf64TGIR(yD'a 4C?3KHe-d(54OfCnR-uvSLtqH,?Ht,HDj8"WtXjHX5wsar1ko.fRXI4f0HKvdl9jjz,i7j'-RBzw"SRwzvThJmuLlSV8VTt-PheAu)knrjNV'])'
GAd.5jUO4lat5af8.N
vPh?FT?PgZ7t3A)j7b1CUIxAbBz.[tKSOzECH eYvbgzj'EgzYQ358vYXonlfFi U1CbvLvfqtph?-3(bVSyv4 

-------------------------------------------------------------------------------------------------------------------


 After 50000 iterations

 I new. Te.ts  
I dids!- 
Bntert me back:]  
[Erd:]  
Then tre there's a back ithin, erom!

, Life is a seep ald I drysfarm... Mescked herus]  
These war untatcor mat...  
[Thosh orguthe sicke-the, verle- thine  
[Anter, prings  
[Gryon of to it  
[there and moss me  
[Brain  
And made out, our the semas ellsed, have it every do